In [ ]:
# !pip install langchain arxiv pymupdf

In [ ]:
# !pip install pdfminer.six

In [1]:
from langchain.document_loaders import PDFMinerLoader

In [2]:

csv_putput_name = "train_coaxnn_paper.csv"
book = PDFMinerLoader(file_path='pdfs/CoAxNN_Optimizing_on-device_deep_learning_with_conditional_approximate_neural_networks.pdf').load()


ValueError: File path pdfs/CoAxNN_Optimizing_on-device_deep_learning_with_conditional_approximate_neural_networks.pdf is not a valid file or url

In [ ]:
type(book)

In [9]:
len(book)

1

In [10]:
all_text = book[0].page_content

In [11]:
# all_text = book[0].page_content.replace("\n", " ")

In [12]:
# ix = 0
# temp = all_text
# while True:
#     print(ix)
#     all_text = all_text.replace("  ", " ")
#     if temp == all_text:
#         break
#     temp = all_text
#     ix += 1

In [59]:
len(all_text)

80987

In [60]:
print(book[0].page_content[10000:10200])

may have different influences on model prediction. Li
et al. [11] proposed a flexible-rate filter pruning approach, FlexPruner,
which automatically selects the number of filters to be pruned for
each 


In [76]:

# all paper without references
paper_text = book[0].page_content[0:-12565]
# print(book[0].page_content[0:-12565])

# references
references_text = book[0].page_content[-12565:]
# print(book[0].page_content[-12565:])


In [231]:
import re

ref_pattern = r'\[\d+\]'
split = re.split(ref_pattern, references_text)
references_list = split[1:-1] + [split[-1].split('.\n')[0]]
references_list = [r.strip() for r in references_list]
references_list = [f"Reference [{i+1}]: "+r for i, r in enumerate(references_list)]
references_list

['Reference [1]: K. He, X. Zhang, S. Ren, J. Sun, Deep residual learning for image recognition, in:\nProceedings of the IEEE Conference on Computer Vision and Pattern Recognition,\n2016, pp. 770–778.',
 'Reference [2]: Y. He, G. Kang, X. Dong, Y. Fu, Y. Yang, Soft filter pruning for accelerating\nin: Proceedings of the Twenty-Seventh\nIntelligence (IJCAI), 2018, pp.\n\ndeep convolutional neural networks,\nInternational Joint Conference on Artificial\n2234–2240.',
 'Reference [3]: S.K. Esser, J.L. McKinstry, D. Bablani, R. Appuswamy, D.S. Modha, Learned\nstep size quantization, in: International Conference on Learning Representations,\n2020.',
 'Reference [4]: Y. Guo, A. Yao, Y. Chen, Dynamic network surgery for efficient dnns,\n\nin:\n\nAdvances in Neural Information Processing Systems, Vol. 29, 2016.',
 'Reference [5]: S. Han, J. Pool, J. Tran, W. Dally, Learning both weights and connections for\nefficient neural network, in: Advances in Neural Information Processing Systems,\nVol. 28

In [232]:
references_list[-1]

'Reference [51]: J. Horn, N. Nafpliotis, D.E. Goldberg, Multiobjective optimization using the\n\nniched Pareto genetic algorithm, 1993'

In [233]:
len(paper_text)

68422

In [235]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1024,
    chunk_overlap  = 300,
    separators = ["\n"]
)

In [236]:
splits = text_splitter.split_text(paper_text)

In [237]:
len(splits)

95

In [238]:
splits[0]

'Contents lists available at ScienceDirect\n\nJournal of Systems Architecture\n\njournal homepage: www.elsevier.com/locate/sysarc\n\nCoAxNN: Optimizing on-device deep learning with conditional approximate\nneural networks\nGuangli Li a,b,1, Xiu Ma c,d,1, Qiuchu Yu a,b, Lei Liu c,d, Huaxiao Liu c,d, Xueying Wang a,b,∗\na State Key Lab of Processors, Institute of Computing Technology, Chinese Academy of Sciences, Beijing, China\nb University of Chinese Academy of Sciences, Beijing, China\nc College of Computer Science and Technology, Jilin University, Changchun, China\nd MOE Key Laboratory of Symbolic Computation and Knowledge Engineering, Jilin University, Changchun, China\n\nA R T I C L E I N F O\n\nA B S T R A C T\n\nKeywords:\nOn-device deep learning\nEfficient neural networks\nModel approximation and optimization\n\nWhile deep neural networks have achieved superior performance in a variety of intelligent applications, the\nincreasing computational complexity makes them difficult to 

In [239]:
splits[1]

'Efficient neural networks\nModel approximation and optimization\n\nWhile deep neural networks have achieved superior performance in a variety of intelligent applications, the\nincreasing computational complexity makes them difficult to be deployed on resource-constrained devices. To\nimprove the performance of on-device inference, prior studies have explored various approximate strategies,\nsuch as neural network pruning, to optimize models based on different principles. However, when combining\nthese approximate strategies, a large parameter space needs to be explored. Meanwhile, different configuration\nparameters may interfere with each other, damaging the performance optimization effect. In this paper, we\npropose a novel model optimization framework, CoAxNN, which effectively combines different approximate\nstrategies, to facilitate on-device deep learning via model approximation. Based on the principles of different'

In [240]:
for s in splits:
    ls = len(s)
    if ls >= 1020:
        print(ls)


1022
1022
1023
1021
1020
1020
1022
1022


In [241]:
import pandas as pd
df = pd.DataFrame([split for split in splits] + references_list, columns=['text'])

In [242]:
df.shape

(146, 1)

In [263]:
df.iloc[10]

text    motivation are introduced in Section 2. The de...
Name: 10, dtype: object

In [244]:
df.head()

,text
0,Contents lists available at ScienceDirect\n\nJ...
1,Efficient neural networks\nModel approximation...
2,"propose a novel model optimization framework, ..."
3,up to 1.53× speedup while reducing energy by u...
4,to the limited resources.\n\nMany efforts have...


In [262]:
df.iloc[-10].values

array(['Reference [42]: Y. He, X. Zhang, J. Sun, Channel pruning for accelerating very deep neural\nnetworks, in: Proceedings of the IEEE International Conference on Computer\nVision, 2017, pp. 1389–1397.'],
      dtype=object)

In [245]:
df.to_csv(csv_putput_name, index=False)

In [246]:
csv_putput_name

'train_coaxnn_paper.csv'

In [257]:
for s in splits:
    print("______________________________________________________________\n" + s + "______________________________________________________________\n\n\n")

______________________________________________________________
Contents lists available at ScienceDirect

Journal of Systems Architecture

journal homepage: www.elsevier.com/locate/sysarc

CoAxNN: Optimizing on-device deep learning with conditional approximate
neural networks
Guangli Li a,b,1, Xiu Ma c,d,1, Qiuchu Yu a,b, Lei Liu c,d, Huaxiao Liu c,d, Xueying Wang a,b,∗
a State Key Lab of Processors, Institute of Computing Technology, Chinese Academy of Sciences, Beijing, China
b University of Chinese Academy of Sciences, Beijing, China
c College of Computer Science and Technology, Jilin University, Changchun, China
d MOE Key Laboratory of Symbolic Computation and Knowledge Engineering, Jilin University, Changchun, China

A R T I C L E I N F O

A B S T R A C T

Keywords:
On-device deep learning
Efficient neural networks
Model approximation and optimization

While deep neural networks have achieved superior performance in a variety of intelligent applications, the
increasing computation